In [64]:
import socket
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [65]:
class Price:
    def __init__(self, feedcode, bid, ask):
        self.feedcode = feedcode
        self.bid = bid # (price, volume)
        self.ask = ask # price, volume)
        self.timestamp = pd.Timestamp.utcnow()

    @classmethod
    def from_packet(cls, packet):
        fields = str(packet).split("|")

        feedcode = fields[1].replace("FEEDCODE=", '')
        bid_price = float(fields[2].replace("BID_PRICE=", ''))
        bid_volume = float(fields[3].replace("BID_VOLUME=", ''))

        ask_price = float(fields[4].replace("ASK_PRICE=", ''))
        ask_volume = float(fields[5].replace("ASK_VOLUME=", '').replace('\'', ''))
        
        return cls(feedcode, (bid_price, bid_volume), (ask_price, ask_volume))
    
    def append_to_df(self, dataframe):
        pass
    
class Trade:
    def __init__(self, feedcode, side, price, volume):
        self.feedcode = feedcode
        self.side = side
        self.price = price
        self.volume = volume
        self.timestamp = pd.Timestamp.utcnow()

    @classmethod
    def from_packet(cls, packet):
        fields = str(packet).split("|")

        feedcode = fields[1].replace("FEEDCODE=", '')
        side = fields[2].replace("SIDE=", '')
        price = float(fields[3].replace("PRICE=", ''))
        volume = float(fields[4].replace("VOLUME=", '').replace('\'', ''))
        
        return cls(feedcode, side, price, volume)
    
    def append_to_df(self, dataframe):
        pass
    

In [77]:
dummy_trade = 'TYPE=TRADE|FEEDCODE=ESX-FUTURE|SIDE=ASK|PRICE=3116|VOLUME=159'
dummy_price = 'TYPE=PRICE|FEEDCODE=SP-FUTURE|BID_PRICE=3002.5|BID_VOLUME=2187|ASK_PRICE=3002.75|ASK_VOLUME=621'

dummy_trade_object = Trade.from_packet(dummy_trade)
print(dummy_trade_object.timestamp, dummy_trade_object.price)

dummy_price_object = Price.from_packet(dummy_price)
print(dummy_price_object.timestamp, dummy_price_object.bid, dummy_price_object.ask)

2019-05-11 17:45:05.461870+00:00 3116.0
2019-05-11 17:45:05.463866+00:00 (3002.5, 2187.0) (3002.75, 621.0)


In [78]:
UDP_IP = "188.166.115.7"
UDP_PORT = 7001
MESSAGE = b"TYPE=SUBSCRIPTION_REQUEST"

sock = socket.socket(socket.AF_INET, # Internet
                      socket.SOCK_DGRAM) # UDP

sock.sendto(MESSAGE, (UDP_IP, UDP_PORT))

i=0

while True:
    packet = sock.recvfrom(65535)
    
    raw_request = str(packet[0])
    
    if "TYPE=PRICE" in raw_request:
        price = Price.from_packet(raw_request)
        print(price.timestamp, price.feedcode, price.bid, price.ask)
    elif "TYPE=TRADE" in raw_request:
        trade = Trade.from_packet(raw_request)
        print(trade.timestamp, trade.side, trade.price, trade.volume)
        
    i+=1


2019-05-11 17:46:49.189193+00:00 ASK 3072.5 1216.0
2019-05-11 17:46:49.317043+00:00 SP-FUTURE (3046.75, 1328.0) (3047.0, 1486.0)
2019-05-11 17:46:49.790387+00:00 ASK 3072.5 8.0
2019-05-11 17:46:49.792376+00:00 SP-FUTURE (3046.75, 1330.0) (3047.0, 1484.0)
2019-05-11 17:46:50.103508+00:00 ASK 3072.5 352.0
2019-05-11 17:46:50.105501+00:00 SP-FUTURE (3046.75, 1813.0) (3047.0, 1001.0)
2019-05-11 17:46:50.260689+00:00 ASK 3047.0 196.0
2019-05-11 17:46:50.261685+00:00 ESX-FUTURE (3072.25, 1263.0) (3072.5, 1692.0)
2019-05-11 17:46:51.292690+00:00 BID 3046.75 249.0
2019-05-11 17:46:51.309651+00:00 ESX-FUTURE (3072.25, 754.0) (3072.5, 2201.0)
2019-05-11 17:46:51.359511+00:00 ASK 3072.5 2200.0
2019-05-11 17:46:51.454710+00:00 SP-FUTURE (3049.5, 1893.0) (3049.75, 921.0)
2019-05-11 17:46:52.203907+00:00 BID 3072.25 99.0
2019-05-11 17:46:52.230831+00:00 SP-FUTURE (3049.5, 1665.0) (3049.75, 1149.0)
2019-05-11 17:46:52.573809+00:00 BID 3049.5 259.0
2019-05-11 17:46:52.596782+00:00 ESX-FUTURE (3072.25,

KeyboardInterrupt: 

1109206
